# <div class="alert alert-block alert-info" style="border-width:4px">Best Model and Last Model </div>

In this notebook, we walk you through the new notion of "best model" vs "last model".

SBrain will keep track of the best model if either of 'StopIfHigherHook' or 'StopIfLowerHook' is used in the model function. Below is an example, where we show case this for a simple MNIST example.

In [ ]:
from sbrain.learning.experiment import *
from sbrain.dataset.dataset import *
import time
import os


def input_function(mode, batch_size, params):
    from tensorflow.examples.tutorials.mnist import input_data
    import tensorflow as tf

    local_dir = "/workspace/shared-dir/sample-notebooks/demo-data/learning/mnist/"

    if mode == "train":
        mnist = input_data.read_data_sets(local_dir, one_hot=True)

        dataset = tf.data.Dataset.from_tensor_slices(({"data" : mnist.train.images}, mnist.train.labels))
        dataset = dataset.shuffle(1000).batch(batch_size).repeat()
        return dataset
    else:
        mnist = input_data.read_data_sets(local_dir, one_hot=True)

        dataset = tf.data.Dataset.from_tensor_slices(({"data" : mnist.test.images}, mnist.test.labels))
        dataset = dataset.batch(batch_size)
        return dataset

In the model function below, please note the use of 'StopIfHigherHook'. If no such hooks are given, SBrain will not keep track of the best model, since SBrain cannot understand what "best" means.

In [ ]:
def my_model_function_with_mnist(features, labels, mode, params):
    import tensorflow as tf

    net = tf.feature_column.input_layer(features, [tf.feature_column.numeric_column("data", shape=(784))])
    # labels = tf.one_hot(labels, 2) ## either or
    for units in [20, 20]:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    # logits = tf.layers.dense(net, 2, activation=None) ## either or
    logits = tf.layers.dense(net, 10, activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)

    # Compute evaluation metrics.
    labels_to_compare = tf.argmax(labels, 1)
    accuracy = tf.metrics.accuracy(labels=labels_to_compare,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    global_step = tf.train.get_global_step()
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss, global_step=global_step)

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op,
                                      training_chief_hooks=None,
                                      training_hooks=[StopIfHigherHook(metric_name="accuracy",
                                                                       threshold=0.50, min_steps=20, run_every_secs=15)]
                                      )

We submit the job and wait for it to finish.

In [ ]:
estimator = Estimator.NewClassificationEstimator(model_fn=my_model_function_with_mnist)
name = "BestModelEstimator" + str(time.time()).replace(".", "")
estimator = Estimator.create(name, "Hello", estimator)

hyper_parameters = HParams(iterations=50000, batch_size=10)
rc = RunConfig(no_of_ps=1, no_of_workers=1, summary_save_frequency=5000, run_eval=True, use_gpu=False, checkpoint_frequency_in_steps=500)

exper = Experiment.run(experiment_name="BestModelEstimator" + str(time.time()).replace(".", ""),
                       description="Really first model",
                       estimator=estimator,
                       hyper_parameters=hyper_parameters,
                       run_config=rc,
                       dataset_version_split=None,
                       input_function=input_function)
job = exper.get_single_job()
print(job.__dict__)
print("")
print("tensorboard url")
print(job.get_tensorboard_url())

job.has_finished()

job.wait_until_finish()

print("Is the job success?? : {}".format(job.is_success()))

print()
print("Model metrics..")
print(job.get_model().model_metrics)

The produced model has an additional method to check if it has a "best model weights". model.has_best_model(). If the training happened without hooks, then this will return false.

In [ ]:
model = job.get_model()
print("Does this model have best model weights? - {}".format(model.has_best_model()))

Also, there is additional boolean attribute 'best_model' added to deploy() method to identify which weights to deploy. Whether it is the best weights or the latest weights. 

If the model does not have best_model weights (if it has not been trained with the hooks), then it will throw an error if the attribute is true.

In [ ]:
me = model.deploy("MyEndpoint" + str(time.time()).replace(".", ""),
                  "myDesc", 1, best_model=True)

In [ ]:
req = np.random.standard_normal((1, 784))
ft = {"features":req}
ac = me.predict(ft)
print(ac)

In [ ]:
latest_model_checkpoint = model.get_model_checkpoint(best_model=False)
best_model_checkpoint = model.get_model_checkpoint(best_model=True)

Cheers!!